In [187]:
import csv 
import pandas as pd
import numpy as np
from datetime import datetime 
import re
from unidecode import unidecode

In [188]:
#Diretórios
DIR_ESCOLAS = r'..\arquivos csv\escolas.csv'
DIR_MATERIAL_DIDATICO = r'..\arquivos csv\material_didatico.csv'
DIR_SUB_PREFEITURAS = r'..\arquivos csv\subprefeituras.csv'

# Importações

In [189]:
#Importações
escolas = pd.read_csv(DIR_ESCOLAS)
material_didatico = pd.read_csv(DIR_MATERIAL_DIDATICO)
sub_prefeituras = pd.read_csv(DIR_SUB_PREFEITURAS)

In [190]:
escolas.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"


In [191]:
material_didatico.head(2)

,id,Quantidade
0,178,20
1,634,121


In [192]:
sub_prefeituras.head(2)

,nome,subprefeitura
0,Santo Cristo,Centro
1,Gamboa,Centro


# Merge

In [193]:
#Quantidade de material didático (md) à ser entregue
escolas_quantidade_md = pd.merge(escolas, material_didatico, how='left', on='id')
escolas_quantidade_md.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",121


In [194]:
#Define os tipos de escolas

def define_tipo(x):
    if 'EM' in x.upper() or 'E.M' in x.upper():
        return 'EM'
    elif 'CIEP' in x.upper() or 'CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA' in x.upper():
        return 'CIEP'
    elif 'ESCOLA' in x.upper():
        return 'ESCOLA'
    else:
        None
    
escolas_quantidade_md['tipo'] = escolas_quantidade_md['Escolas_Postos'].apply(lambda x: define_tipo(x))
escolas_quantidade_md.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",20,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",121,EM


# Tratamentos

In [195]:
# Converte todos os valores strings para UPPER CASE
escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: x.upper() if isinstance(x, str) else x)
# Remove acentuações
escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
escolas_quantidade_md.head(2)

C:\Users\Danie\AppData\Local\Temp\ipykernel_16060\3799985094.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: x.upper() if isinstance(x, str) else x)
C:\Users\Danie\AppData\Local\Temp\ipykernel_16060\3799985094.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)


,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326",20,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121,EM


In [196]:
# Acesse o nome da coluna na posição 4 (índice 3)
coluna_na_posicao_4 = escolas_quantidade_md.columns[3]

# Imprima o nome da coluna na posição 3
print(coluna_na_posicao_4) # indica que "ENDEREÇO" é "ENDEREÇO "

ENDEREÇO 


In [197]:
# Função para substituir palavras em texto da coluna 'ENDEREÇO '
def substituir_palavras(texto, substituicoes):
    for palavra, substituta in substituicoes.items():
        texto = texto.replace(palavra, substituta)
    return texto

substituicoes_endereco = {
    '.': '',
    ',': '',
    'ESTR ': 'ESTRADA ',
    'JR ': 'JUNIOR ',
    'AV ': 'AVENIDA ',
    'MAL ': 'MARECHAL ',
    'ENG ': 'ENGENHEIRO ',
    'SD ': 'SOLDADO ',
    'PCA ': 'PRACA ',
    'ALM ': 'ALMIRANTE ',
    'SEN ': 'SENADOR ',
    'CONJ ': 'CONJUNTO ',
}

# Define um padrão para encontrar "R" como uma palavra independente em endereco para substituição
padrao_r = r'\bR\b'

# Aplica as funções de substituição à coluna endereco
escolas_quantidade_md['ENDEREÇO '] = escolas_quantidade_md['ENDEREÇO '].apply(substituir_palavras, args=(substituicoes_endereco,))
escolas_quantidade_md['ENDEREÇO '] = escolas_quantidade_md['ENDEREÇO '].str.replace(padrao_r, 'RUA', regex=True)

# Função para extrair complementos do endereço
#def extrair_complemento(endereco):
#    complementos = re.findall(r'\b(BLOCO|CONJUNTO)\s+(\S+)\b', endereco, flags=re.IGNORECASE)
#    if complementos:
#        return ','.join([f'{comp[0]} {comp[1]}' for comp in complementos])
#    else:
#        return "SEM COMPLEMENTO"
    
def extrair_complemento(endereco):
    complementos = re.findall(r'\b(BLOCO|CONJUNTO)\s+(\S+)\b', endereco, flags=re.IGNORECASE)
    
    if complementos:
        complemento_str = ','.join([f'{comp[0]} {comp[1]}' for comp in complementos])
        endereco_limpo = re.sub(r'\b(BLOCO|CONJUNTO)\s+(\S+)\b', '', endereco, flags=re.IGNORECASE)
        return complemento_str, endereco_limpo.strip()
    else:
        return "SEM COMPLEMENTO", endereco

escolas_quantidade_md['ENDEREÇO '] = escolas_quantidade_md['ENDEREÇO '].apply(extrair_complemento)

# Função para extrair números do endereço
def extrair_numero(endereco):
    numeros = re.findall(r'\d+', endereco)
    if numeros:
        return '-'.join(numeros)
    else:
        return 0

# Aplica a função de extração e cria uma nova coluna 'numero_rua'
escolas_quantidade_md['numero_rua'] = escolas_quantidade_md['ENDEREÇO '].apply(extrair_numero)

# Função para remover números e variações do endereço
def remover_numeros_variacoes(endereco):
    padrao = r'(\d+|S/No|S/Ndeg|S/N)'
    endereco_limpo = re.sub(padrao, '', endereco).strip()
    return endereco_limpo

# Aplica a função de remoção e renomeia a coluna
escolas_quantidade_md['ENDEREÇO'] = escolas_quantidade_md['ENDEREÇO '].apply(remover_numeros_variacoes)
#escolas_quantidade_md.rename(columns={'ENDEREÇO ': 'logradouro_da_entrega'}, inplace=True)

TypeError: expected string or bytes-like object

In [ ]:
print(escolas_quantidade_md)
escolas_quantidade_md.to_csv('df_escolas_modificado.csv', index=False)

      id                               Escolas_Postos             BAIRRO  \
0    178  CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL               CAJU   
1    634                   EM ALICE DO AMARAL PEIXOTO            BENFICA   
2    483                           EM CELESTINO SILVA             CENTRO   
3    476            ESCOLA MUNICIPAL FLORIANO PEIXOTO      SAO CRISTOVAO   
4    132                            EM PEREIRA PASSOS       RIO COMPRIDO   
..   ...                                          ...                ...   
147  474                  EM ENGENHEIRO GASTAO RANGEL          GUARATIBA   
148  301                           EM JONATAS SERRANO          GUARATIBA   
149  215                          E.M. NARCISA AMALIA  ILHA DE GUARATIBA   
150  606                        EM PROFESSOR CASTILHO  ILHA DE GUARATIBA   
151   69           ESCOLA MUNICIPAL CARDEAL ARCOVERDE          MADUREIRA   

                              ENDEREÇO          lat         lon Quantidade  \
0        

# Subprefeituras


In [ ]:
# Converte todos os valores strings para UPPER CASE
sub_prefeituras_md = sub_prefeituras.applymap(lambda x: x.upper() if isinstance(x, str) else x)
sub_prefeituras_md

# Remove acentuações
sub_prefeituras_md = sub_prefeituras_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
sub_prefeituras_md

# Teste
escolas_quantidade_md = escolas_quantidade_md.merge(sub_prefeituras_md, left_on='BAIRRO', right_on='nome', how='left')
escolas_quantidade_md.drop(columns=['nome'], inplace=True)
escolas_quantidade_md


C:\Users\Danie\AppData\Local\Temp\ipykernel_16060\1392222361.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_prefeituras_md = sub_prefeituras.applymap(lambda x: x.upper() if isinstance(x, str) else x)
C:\Users\Danie\AppData\Local\Temp\ipykernel_16060\1392222361.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_prefeituras_md = sub_prefeituras_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)


,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo,ENDEREÇO_1,numero_rua,ENDEREÇO_2,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326",20,CIEP,SEM COMPLEMENTO,0,RUA CARLOS SEIDL,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121,EM,SEM COMPLEMENTO,0,RUA EBANO,CENTRO
2,483,EM CELESTINO SILVA,CENTRO,RUA DO LAVRADIO 56,"-22,909293","-43,183579",220,EM,SEM COMPLEMENTO,0,RUA DO LAVRADIO,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456",190,ESCOLA,SEM COMPLEMENTO,0,PRACA ARGENTINA,CENTRO
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579",67,EM,SEM COMPLEMENTO,0,PRACA CONDESSA PAULO DE FRONTIN,CENTRO
...,...,...,...,...,...,...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9183,"-22,98046","-43,643545",320,EM,SEM COMPLEMENTO,0,ESTRADA DO MAGARCA,ZONA OESTE
148,301,EM JONATAS SERRANO,GUARATIBA,ESTRADA DO MATO ALTO S/No,"-22,953163","-43,577409",335,EM,SEM COMPLEMENTO,0,ESTRADA DO MATO ALTO,ZONA OESTE
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/No,"-23,009084","-43,537582",NaN,EM,SEM COMPLEMENTO,0,ESTRADA TEODORETO DE CAMARGO,ZONA OESTE
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,"-22,994124","-43,593683",521,EM,SEM COMPLEMENTO,0,CAMINHO DA MATRIZ,ZONA OESTE
